In [91]:
import nltk
import random
from nltk.corpus import movie_reviews

# http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/

In [92]:
documents = [(list(movie_reviews.words(fileid)), category)
                for category in movie_reviews.categories()
                    for fileid in movie_reviews.fileids(category)]

In [93]:
# random.shuffle(documents) # The first 1000 negative and the second half, 1000, positive.

In [94]:
all_words = []
for w in movie_reviews.words():
    all_words.append(w.lower())

all_words_dist = nltk.FreqDist(all_words)

In [95]:
from nltk.corpus import stopwords
import string

stop_words = set(stopwords.words("english"))
stop_words = stop_words.union(set(string.punctuation))

# print(string.punctuation)

filtered_words = []
for w in all_words:
    if w not in stop_words:
        filtered_words.append(w)
        
filtered_words_dist = nltk.FreqDist(filtered_words)

In [96]:
word_features = list(filtered_words_dist.keys())[:3000]
###############################
# BUGBUG ??
###############################
word_features = filtered_words_dist.most_common(3000)
word_features = [word_feature[0]
                    for word_feature in word_features]
word_features.sort()

In [97]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features: # Why not enumerating words in the document? --> Then no way to find "false" words.
        features[w] = (w in words) # True if w is in words/the document.
        
    return features

In [98]:
featuresets = [(find_features(rev), category) 
                   for (rev, category) in documents]

In [99]:
# training_set = featuresets[:1900]
# testing_set = featuresets[1900:]

training_set = featuresets[:100] # All the training set are negative, no way to learn from positive data. 
testing_set = featuresets[100:] # Test set is mixed. 

classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

Naive Bayes Algo accuracy percent: 47.368421052631575
Most Informative Features


In [100]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

In [101]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:455: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


MNB_classifier accuracy percent: 47.368421052631575


In [102]:
# GaussianNB_classifier = SklearnClassifier(GaussianNB())
# GaussianNB_classifier.train(training_set)
# print("GaussianNB_classifier accuracy percent:", (nltk.classify.accuracy(GaussianNB_classifier, testing_set))*100)

In [103]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:455: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


BernoulliNB_classifier accuracy percent: 47.368421052631575


In [105]:
# LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
# LogisticRegression_classifier.train(training_set)
# print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

In [106]:
# SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
# SGDClassifier_classifier.train(training_set)
# print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

ValueError: The number of class labels must be greater than one.

In [107]:
SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

ValueError: The number of classes has to be greater than one; got 1

In [108]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

In [110]:
from nltk.classify import ClassifierI
from statistics import mode

class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers): # don't forget *
        self._classifiers = classifiers
        
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes) # pick the most popular one. pos or neg
    
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        
        choice_votes = votes.count(mode(votes)) # the number of most popular one
        conf = choice_votes / len(votes)        # the ratio of the most popular one.
        
        return conf

In [111]:
voted_classifier = VoteClassifier(classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,                                   
                                  LinearSVC_classifier,
                                  SVC_classifier,
                                  NuSVC_classifier, 
                                  LogisticRegression_classifier)

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)

NotFittedError: This LinearSVC instance is not fitted yet

In [112]:
        print("Classification", voted_classifier.classify(testing_set[0][0]), ", Confidence Score", voted_classifier.confidence(testing_set[0][0])*100, "%" )

NotFittedError: This LinearSVC instance is not fitted yet

In [113]:
print("Classification", voted_classifier.classify(testing_set[1][0]), ", Confidence Score", voted_classifier.confidence(testing_set[1][0])*100, "%" )
print("Classification", voted_classifier.classify(testing_set[2][0]), ", Confidence Score", voted_classifier.confidence(testing_set[2][0])*100, "%" )
print("Classification", voted_classifier.classify(testing_set[3][0]), ", Confidence Score", voted_classifier.confidence(testing_set[3][0])*100, "%" )
print("Classification", voted_classifier.classify(testing_set[4][0]), ", Confidence Score", voted_classifier.confidence(testing_set[4][0])*100, "%" )
print("Classification", voted_classifier.classify(testing_set[5][0]), ", Confidence Score", voted_classifier.confidence(testing_set[5][0])*100, "%" )
print("Classification", voted_classifier.classify(testing_set[6][0]), ", Confidence Score", voted_classifier.confidence(testing_set[6][0])*100, "%" )
print("Classification", voted_classifier.classify(testing_set[7][0]), ", Confidence Score", voted_classifier.confidence(testing_set[7][0])*100, "%" )

NotFittedError: This LinearSVC instance is not fitted yet